# <div align="center">🌟 Проект: "Neural network" </div>
# <div align="center">Создаём аналог PyTorch с нуля на NumPy & SciPy </div>

## Содержание

1. [Импорт библиотек NumPy & SciPy](##Импорт-библиотек-NumPy-&-SciPy)
2. [Базовый класс для всех модулей нейронной сети](##Базовый-класс-для-всех-модулей-нейронной-сети)
3. [Sequential](##Sequential)
4. [Полносвязный слой (Linear Transform Layer)](##Linear-Transform-Layer-(Fully-Connected-Layer))
5. [Flatten](##Flatten)
6. [Softmax + LogSoftMax](##Softmax-+-LogSoftMax)
7. [Нормализация по батчам и масштабирование по каналам (Batch Normalization + Channelwise Scaling)](##Batch-Normalization-+-ChannelwiseScaling)
8. [Dropout](##Dropout)
9. [ReLU](##ReLU)
10. [Классы критериев (NLLCriterion и MSECriterion)](##Criterion)
11. [Оптимизаторы (Adam и SGD)](##Optimizers)
12. [Двумерный сверточный слой (Conv2D)](##Conv2D---Двумерный-сверточный-слой)
13. [Максимальный пуллинг (MaxPool2d)](##MaxPool2d)

## Импорт библиотек NumPy & SciPy

In [22]:
import numpy as np
import scipy as sp
import scipy.signal

## Базовый класс для всех модулей нейронной сети

**Module** — это базовый класс для всех модулей нейронной сети. Он представляет собой абстракцию, которая позволяет работать с различными компонентами нейронной сети, такими как слои, функции активации, регуляризации и т.д. Модуль отвечает за вычисления в процессе обучения и тестирования, а также за выполнение шагов обратного распространения ошибки (backpropagation) для обновления параметров сети.

In [3]:
class Module(object):
    """
    Базовый класс для всех модулей нейронной сети.

    По сути, модуль можно рассматривать как нечто (черный ящик),
    что может обрабатывать входные данные и производить выходные данные.
    """
    def __init__ (self):
        self.output = None  # Выходные данные модуля
        self.gradInput = None # Градиент по входу модуля (dLoss/dInput)
        self.training = True # Флаг, указывающий, находится ли модуль в режиме обучения

    def forward(self, input):
        """
        Принимает входной объект и вычисляет соответствующий выход модуля.
        """
        return self.updateOutput(input)

    def backward(self, input, gradOutput):
        """
        Выполняет шаг обратного распространения через модуль по отношению к заданному входу.

        Это включает в себя:
         - вычисление градиента по отношению к входу (необходимо для дальнейшего обратного распространения),
         - вычисление градиента по отношению к параметрам (для обновления параметров во время оптимизации).
        """
        self.updateGradInput(input, gradOutput)  # Вычисляем градиент по входу
        self.accGradParameters(input, gradOutput) # Аккумулируем градиенты по параметрам
        return self.gradInput # Возвращаем градиент по входу

    def updateOutput(self, input):
        """
        Вычисляет выход, используя текущий набор параметров класса и входные данные.
        Эта функция возвращает результат, который сохраняется в поле output.
        """
        pass

    def updateGradInput(self, input, gradOutput):
        """
        Вычисляет градиент модуля по отношению к его собственному входу.
        Это возвращается в gradInput. Кроме того, соответствующим образом обновляется переменная состояния gradInput.
        """
        pass

    def accGradParameters(self, input, gradOutput):
        """
        Вычисляет градиент модуля по отношению к его собственным параметрам.
        """
        pass

    def zeroGradParameters(self):
        """
        Обнуляет переменную gradParams, если у модуля есть параметры.
        """
        pass

    def getParameters(self):
        """
        Возвращает список со своими параметрами.
        Если у модуля нет параметров, возвращает пустой список.
        """
        return []

    def getGradParameters(self):
        """
        Возвращает список с градиентами по отношению к его параметрам.
        Если у модуля нет параметров, возвращает пустой список.
        """
        return []

    def train(self):
        """
        Устанавливает режим обучения для модуля.
        Поведение во время обучения и тестирования различается для Dropout и BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Устанавливает режим оценки для модуля.
        """
        self.training = False

    def __repr__(self):
        """
        Красивая печать.
        """
        return "Module"

## Sequential

**Sequential** — это контейнер, который позволяет строить нейронные сети путём последовательного добавления слоев. Все слои в контейнере выполняются один за другим, и выход одного слоя становится входом для следующего. Это делает `Sequential` удобным для построения моделей, где слои применяются по порядку, без необходимости явного указания связи между ними.

В отличие от функциональных или более сложных моделей, где слои могут быть соединены произвольным образом (например, через skip-соединения или другие нестандартные связи), `Sequential` предполагает строгое последовательное выполнение операций.


In [4]:
class Sequential(Module):
    """
         Этот класс реализует контейнер, который последовательно обрабатывает входные данные.

         Входные данные обрабатываются каждым модулем (слоем) в self.modules последовательно.
         Полученный массив называется выходными данными.
    """

    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []

    def add(self, module):
        """
        Добавляет модуль в контейнер.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Основной алгоритм ПРЯМОГО ПРОХОДА:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})

        """
        self.output = input  # Начальное значение выхода - это входные данные
        for module in self.modules: # Итерируемся по всем модулям
            self.output = module.forward(self.output) # Выход предыдущего модуля становится входом для следующего
        return self.output # Возвращаем выход последнего модуля

    def backward(self, input, gradOutput):
        """
        Алгоритм ОБРАТНОГО ПРОХОДА:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)
        """

        activations = [input] # Список активаций для каждого слоя (включая входной)
        current_input = input

        for module in self.modules: # Прямой проход для сохранения всех активаций
            current_input = module.forward(current_input)
            activations.append(current_input)

        current_gradOutput = gradOutput # Начальный градиент на выходе
        for i in reversed(range(len(self.modules))): # Итерируемся в обратном порядке по модулям
                        current_gradOutput = self.modules[i].backward(activations[i], current_gradOutput) # Вычисляем градиент для текущего модуля и передаем его на предыдущий

        self.gradInput = current_gradOutput # Градиент на входе Sequential модуля - это градиент на выходе первого слоя
        return self.gradInput

    def zeroGradParameters(self):
        for module in self.modules: # Обнуляем градиенты параметров каждого модуля
            module.zeroGradParameters()

    def getParameters(self):
        """
        Собирает все параметры в список.
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Собирает все градиенты по отношению к параметрам в список.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules]) # Представление Sequential модуля в виде строки
        return string

    def __getitem__(self,x):
        return self.modules.__getitem__(x) # Получение модуля по индексу

    def train(self):
        """
        Распространяет параметр training через все модули
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Распространяет параметр training через все модули
        """
        self.training = False
        for module in self.modules:
            module.evaluate()


## Linear Transform Layer (Fully Connected Layer)

Модуль, реализующий линейное преобразование для нейронных сетей. Этот слой принимает входные данные формы `(n_samples, n_features)` и выполняет линейное преобразование, используя матрицу весов и смещения. Линейные слои широко используются в нейронных сетях для выполнения простых, но мощных вычислений, таких как прогнозирование, классификация и регрессия.

In [5]:
class Linear(Module):
    """
    Модуль, реализующий линейное преобразование.
    Модуль работаeт с 2D входом формы (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        """
        Инициализирует линейный слой.

        Args:
            n_in (int): Количество входных признаков.
            n_out (int): Количество выходных признаков.
        """
        super(Linear, self).__init__()

        stdv = 1./np.sqrt(n_in)  # Вычисляем стандартное отклонение для инициализации
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in)) # Матрица весов (n_out x n_in), инициализирована случайными значениями из равномерного распределения
        self.b = np.random.uniform(-stdv, stdv, size = n_out) # Вектор смещений (n_out), инициализирован аналогично

        self.gradW = np.zeros_like(self.W) # Градиент по весам (имеет ту же форму, что и W), инициализирован нулями
        self.gradb = np.zeros_like(self.b) # Градиент по смещениям (имеет ту же форму, что и b), инициализирован нулями

    def updateOutput(self, input):
        """
        Вычисляет выход линейного слоя.

        Args:
            input (np.ndarray): Входные данные формы (n_samples, n_in).

        Returns:
            np.ndarray: Выходные данные формы (n_samples, n_out).
        """
        self.output = np.dot(input, self.W.T) + self.b  # Линейное преобразование: input @ W.T + b
        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Вычисляет градиент по входу.

        Args:
            input (np.ndarray): Входные данные формы (n_samples, n_in).
            gradOutput (np.ndarray): Градиент на выходе формы (n_samples, n_out).

        Returns:
            np.ndarray: Градиент по входу формы (n_samples, n_in).
        """
        self.gradInput = np.dot(gradOutput, self.W) # gradOutput @ W
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        """
        Аккумулирует градиенты по параметрам (весам и смещениям).

        Args:
            input (np.ndarray): Входные данные формы (n_samples, n_in).
            gradOutput (np.ndarray): Градиент на выходе формы (n_samples, n_out).
        """
        self.gradW += np.dot(gradOutput.T, input) # gradOutput.T @ input
        self.gradb += np.sum(gradOutput, axis=0) # Сумма gradOutput по всем примерам

    def zeroGradParameters(self):
        """
        Обнуляет градиенты по параметрам (весам и смещениям).
        """
        self.gradW.fill(0) # Заполняем gradW нулями
        self.gradb.fill(0) # Заполняем gradb нулями

    def getParameters(self):
        """
        Возвращает параметры (веса и смещения).

        Returns:
            list: Список, содержащий матрицу весов W и вектор смещений b.
        """
        return [self.W, self.b]

    def getGradParameters(self):
        """
        Возвращает градиенты по параметрам (градиент по весам и градиент по смещениям).

        Returns:
            list: Список, содержащий градиент по весам gradW и градиент по смещениям gradb.
        """
        return [self.gradW, self.gradb]

    def __repr__(self):
        """
        Возвращает строковое представление объекта Linear.
        """
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0]) # Форматируем строку с информацией о слое
        return q

## Flatten

Класс `Flatten` реализует операцию преобразования входных данных в одномерный массив, сохраняя количество элементов. Он используется для преобразования многомерных тензоров в одномерные в процессе построения нейронных сетей, особенно перед полносвязными слоями.


In [6]:
class Flatten(Module):
    def __init__(self):
         super(Flatten, self).__init__()
    
    def updateOutput(self, input):
        self.output = input.reshape(len(input), -1)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput
    
    def __repr__(self):
        return "Flatten"

## Softmax + LogSoftMax

Слой Softmax преобразует вектор "сырых" оценок (логитов) в вектор вероятностей, где каждый элемент отражает вероятность принадлежности к определенному классу. Используется в задачах многоклассовой классификации.

Операция:

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$


Вход: (batch_size, n_feats) - Пакет векторов, где каждый вектор - "сырые" оценки для примера.

Выход: (batch_size, n_feats) - Пакет вероятностных распределений (сумма элементов каждой строки = 1).

In [7]:
class SoftMax(Module):
    """
    Модуль, реализующий функцию Softmax.
    """
    def __init__(self):
        """
        Инициализирует слой Softmax.
        """
        super(SoftMax, self).__init__()

    def updateOutput(self, input):
        """
        Вычисляет выход слоя Softmax.

        Args:
            input (np.ndarray): Входные данные формы (batch_size, n_feats) - "сырые" оценки.

        Returns:
            np.ndarray: Выходные данные формы (batch_size, n_feats) - вероятности.
        """
        self.output = np.subtract(input, input.max(axis=1, keepdims=True)) # Вычитаем максимальное значение из каждой строки для избежания переполнения при вычислении экспоненты

        exp_x_i = np.exp(self.output)  # Вычисляем экспоненту для каждого элемента
        self.output = exp_x_i / np.sum(exp_x_i, axis=1, keepdims=True) # Нормализуем: делим на сумму экспонент по каждой строке, чтобы получить вероятности
        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Вычисляет градиент по входу для слоя Softmax.

        Args:
            input (np.ndarray): Входные данные формы (batch_size, n_feats) - "сырые" оценки.
            gradOutput (np.ndarray): Градиент на выходе формы (batch_size, n_feats).

        Returns:
            np.ndarray: Градиент по входу формы (batch_size, n_feats).
        """
        self.gradInput = np.zeros_like(input) # Инициализируем градиент нулями, той же формы что и вход
        for i in range(self.output.shape[0]): # Итерируемся по каждому примеру в батче
            jacobian = np.diag(self.output[i]) - np.outer(self.output[i], self.output[i]) # Вычисляем якобиан для каждого примера
            self.gradInput[i] = np.dot(jacobian, gradOutput[i]) # Вычисляем градиент по входу, используя якобиан и градиент на выходе

        return self.gradInput

    def __repr__(self):
        """
        Возвращает строковое представление объекта SoftMax.
        """
        return "SoftMax"

**LogSoftMax**

- **Входные данные**: **`batch_size x n_feats`**
- **Выходные данные**: **`batch_size x n_feats`**

Формула для расчета log-softmax для каждого элемента:

$$
\text{logsoftmax}(x)_i = \log(\text{softmax}(x)_i) = x_i - \log \left( \sum_j \exp(x_j) \right)
$$

Слой `LogSoftmax` используется для числовой стабильности при вычислениях логарифма вероятностей в многоклассовых задачах классификации.

In [ ]:
class LogSoftmax(Module):
    def __init__(self):
         super(LogSoftmax, self).__init__()
    
    def updateOutput(self, input):
        # Нормализация для числовой стабильности (вычитание максимального значения по каждому ряду)
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Рассчитываем LogSumExp для стабилизации чисел при вычислениях
        log_sum_exp = np.log(np.sum(np.exp(self.output), axis=1, keepdims=True))
        
        # Вычисляем логарифм вероятности softmax
        self.output = self.output - log_sum_exp
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Получаем размерность входных данных
        batch_size, n_features = input.shape
        
        # Вычисляем вероятности softmax на основе выходных значений
        softmax_probs = np.exp(self.output)

        # Суммируем градиенты по всем элементам в каждом ряду
        sum_gradOutput = np.sum(gradOutput, axis=1, keepdims=True)
        
        # Рассчитываем градиенты для входных данных с использованием градиентов выходных данных
        self.gradInput = gradOutput - softmax_probs * sum_gradOutput
        
        return self.gradInput
    
    def __repr__(self):
        return "LogSoftMax"


## Batch Normalization + ChannelwiseScaling

Одним из наиболее значимых нововведений, оказавших огромное влияние на нейронные сети, является Batch Normalization. Идея проста, но эффективна: признаки должны быть отбелены ($mean = 0$, $std = 1$) на протяжении всей нейронной сети. Это улучшает сходимость глубоких моделей, позволяя обучать их днями, а не неделями. 

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

Принцип работы:

Во время обучения (`self.training == True`) он преобразует входные данные как:

$$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$

где:

•  μ - среднее значение признаков в батче.\
•  σ^2 - дисперсия признаков в батче.\
•  ϵ - небольшое число для численной стабильности (чтобы избежать деления на ноль).\

Также, во время обучения, слой должен поддерживать экспоненциально скользящие средние значения для среднего и дисперсии:

```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```

где:

•  alpha - коэффициент сглаживания\
•  batch_mean - среднее значение признаков в текущем батче\
•  batch_variance - дисперсия признаков в текущем батче\
•  self.moving_mean - Экспоненциальное скользящее среднее среднего значения\
•  self.moving_variance - Экспоненциальное скользящее среднее дисперсии\

Во время тестирования (self.training == False) слой нормализует входные данные, используя moving_mean и moving_variance.

В общем случае, "batch normalization" всегда подразумевает нормализацию + масштабирование.

Но маштабирование организовано отдельно в ChannelwiseScaling.

После того как данные нормализованы с помощью BatchNormalization, может потребоваться их масштабирование и смещение с использованием ChannelwiseScaling для восстановления подходящих значений, что поможет улучшить производительность модели.

In [8]:
class BatchNormalization(Module):
    """
    Реализация слоя Batch Normalization.

    Этот слой нормализует входные данные для каждой мини-партии, чтобы ускорить обучение
    и сделать его более стабильным.

    Атрибуты:
        EPS (float): Небольшое число, добавляемое к дисперсии для избежания деления на ноль.
        alpha (float): Коэффициент экспоненциального скользящего среднего для обновления
                        статистики (среднее и дисперсия).
        moving_mean (numpy.ndarray): Скользящее среднее входных данных.
        moving_variance (numpy.ndarray): Скользящая дисперсия входных данных.
    """
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        """
        Инициализирует слой Batch Normalization.

        Аргументы:
            alpha (float): Коэффициент экспоненциального скользящего среднего.
                           Значение 0.0 означает, что статистика мгновенно обновляется
                           на основе текущей мини-партии.  Значение близкое к 1.0
                           указывает на более медленное обновление статистики.
        """
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None  # Инициализируется в updateOutput
        self.moving_variance = None # Инициализируется в updateOutput

    def updateOutput(self, input):
        """
        Вычисляет выходные данные слоя для данного входа.

        Аргументы:
            input (numpy.ndarray): Входные данные (мини-партии).

        Возвращает:
            numpy.ndarray: Нормализованные выходные данные.
        """
        self.input_shape = input.shape

        if self.training:
            mean = np.mean(input, axis=0, keepdims=True) # Среднее по каждому признаку
            variance = np.var(input, axis=0, keepdims=True) # Дисперсия по каждому признаку

            # Инициализируем скользящие средние и дисперсии, если они еще не были
            if self.moving_mean is None:
                self.moving_mean = np.zeros_like(mean)
            if self.moving_variance is None:
                self.moving_variance = np.ones_like(variance) #  Начинаем с дисперсии равной 1

            # Обновляем скользящие средние и дисперсии
            self.moving_mean = self.alpha * self.moving_mean + (1 - self.alpha) * mean
            self.moving_variance = self.alpha * self.moving_variance + (1 - self.alpha) * variance


            # Центрируем данные (вычитаем среднее)
            self.x_centered = input - mean
            # Вычисляем стандартное отклонение (корень из дисперсии + EPS)
            self.std = np.sqrt(variance + self.EPS)
            # Нормализуем данные (делим на стандартное отклонение)
            self.x_normalized = self.x_centered / self.std
            self.output = self.x_normalized
        else:
            # Режим инференса: используем скользящие средние и дисперсии.
            x_centered_eval = input - self.moving_mean
            std_eval = np.sqrt(self.moving_variance + self.EPS)
            self.output = x_centered_eval / std_eval

        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Вычисляет градиент входных данных.

        Аргументы:
            input (numpy.ndarray): Входные данные (мини-партии).
            gradOutput (numpy.ndarray): Градиент выходных данных.

        Возвращает:
            numpy.ndarray: Градиент входных данных.
        """
        if not self.training:
            # Режим инференса: градиент вычисляется с использованием скользящих средних и дисперсий.
            std_eval = np.sqrt(self.moving_variance + self.EPS)
            self.gradInput = gradOutput / std_eval
            return self.gradInput

        batch_size = input.shape[0]

        grad_x_normalized = gradOutput

        # Вычисляем градиенты относительно дисперсии и среднего.
        grad_variance = np.sum(grad_x_normalized * self.x_centered, axis=0, keepdims=True) * (-0.5) * np.power(self.std, -3)

        grad_mean = np.sum(grad_x_normalized * (-1.0 / self.std), axis=0, keepdims=True)

        # Комбинируем градиенты для получения градиента входных данных.
        # Разложение производной  d(выхода BN)/d(входа) по правилу цепи:
        # d(x_norm)/dx = 1/std
        # d(среднего)/dx = 1/N
        # d(дисперсии)/dx = 2*(x-среднее)/N
        term1 = grad_x_normalized / self.std  # Часть градиента от нормализации
        term2 = grad_mean / batch_size         # Часть градиента от среднего
        term3 = grad_variance * (2.0 * self.x_centered / batch_size)  # Часть градиента от дисперсии

        self.gradInput = term1 + term2 + term3
        return self.gradInput

    def __repr__(self):
        """
        Возвращает строковое представление слоя.
        """
        return "BatchNormalization"

**ChannelwiseScaling** — это слой, который реализует линейное преобразование входных данных по каналам: 

$$ y = \gamma \cdot x + \beta $$

где:
- $\gamma$ — вектор масштабирования для каждого канала (обучаемый параметр),
- $\beta$ — вектор сдвига для каждого канала (обучаемый параметр),
- $x$ — входные данные (тензор).

Этот слой позволяет каждому каналу входного тензора масштабироваться на свой собственный параметр $\gamma$ и сдвигаться на свой собственный параметр $\beta$. Это дает модели возможность изучать оптимальное масштабирование и сдвиг для каждого канала данных.


In [9]:
class ChannelwiseScaling(Module):
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()
        self.gamma = np.ones(n_out)  # Инициализация gamma
        self.beta = np.zeros(n_out)  # Инициализация beta

    def updateOutput(self, input):
        """
        Вычисляет выходные данные слоя.

        Аргументы:
            input (numpy.ndarray): Входные данные.

        Возвращает:
            numpy.ndarray: Выходные данные.
        """
        # Broadcasting применяется автоматически: gamma и beta добавляются к каждому элементу по всем измерениям, кроме последнего.
        self.output = input * self.gamma.reshape(1, -1, 1, 1) + self.beta.reshape(1, -1, 1, 1)
        return self.output

    def updateGradInput(self, input, gradOutput):
        """
        Вычисляет градиент по входу.

        Аргументы:
            input (numpy.ndarray): Входные данные.
            gradOutput (numpy.ndarray): Градиент выходных данных.

        Возвращает:
            numpy.ndarray: Градиент по входу.
        """
        self.gradInput = gradOutput * self.gamma.reshape(1, -1, 1, 1)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        """
        Вычисляет градиенты по параметрам (gamma и beta).

        Аргументы:
            input (numpy.ndarray): Входные данные.
            gradOutput (numpy.ndarray): Градиент выходных данных.
        """
        self.gradGamma = np.sum(gradOutput * input, axis=(0, 2, 3))  # Суммируем по батчу и пространственным измерениям
        self.gradBeta = np.sum(gradOutput, axis=(0, 2, 3))  # Суммируем по батчу и пространственным измерениям

    def zeroGradParameters(self):
        """
        Обнуляет градиенты параметров (gamma и beta).
        """
        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def getParameters(self):
        """
        Возвращает параметры слоя (gamma и beta).
        """
        return [self.gamma, self.beta]

    def getGradParameters(self):
        """
        Возвращает градиенты параметров слоя (gradGamma и gradBeta).
        """
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        """
        Возвращает строковое представление слоя.
        """
        return "ChannelwiseScaling"

## Dropout

**Dropout** — это техника регуляризации, которая помогает предотвратить переобучение нейронной сети. Идея заключается в том, чтобы случайным образом "выключать" (обнулять) определенные элементы вектора входных данных во время обучения. Это позволяет избежать слишком сильной зависимости нейронов друг от друга, улучшая обобщающую способность модели.

Когда модель обучается (`self.training == True`), для каждого батча генерируется маска с вероятностью $p$, где каждый элемент будет равен 0 с вероятностью $p$, и 1 с вероятностью $1 - p$. Для этого используется распределение Бернулли. Это приводит к тому, что элементы входных данных, умноженные на эту маску, обнуляются (т.е. "выключаются"). Для корректного масштабирования данных на этапе тестирования, результаты выходного слоя необходимо умножить на коэффициент $1 / (1 - p)$, чтобы средние значения признаков оставались приближенными к тем, что будут на этапе тестирования, где все нейроны активны.

В тестовом режиме (`self.training == False`) дроп-аут не применяется, и выходные данные остаются без изменений, т.е. выполняется тождественное преобразование входных данных.


In [10]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        # Вероятность обнуления
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        # Метод обновления выходных данных (выходного тензора)
        if self.training:
            # Создаем маску случайным образом: элементы будут равны 0 с вероятностью p, и 1 с вероятностью (1 - p).
            # Маска масштабируется на (1.0 - p), чтобы сохранить ожидаемое значение выходных данных.
            self.mask = (np.random.rand(*input.shape) > self.p) / (1.0 - self.p)
            # Применяем маску к входным данным
            self.output = input * self.mask
        else:
            # В режиме inference (тестирования), маска не используется, выходные данные остаются без изменений
            self.mask = None
            self.output = input
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Метод обновления градиентов на вход (градиенты по входным данным)
        if self.training:
            # В тренировочном режиме, градиенты передаются только через активные элементы (маска)
            self.gradInput = gradOutput * self.mask
        else:
            # В режиме inference, градиенты передаются без изменений
            self.gradInput = gradOutput
        return self.gradInput
        
    def __repr__(self):
        # Представление объекта класса Dropout
        return "Dropout"

## ReLU

Класс `ReLU` реализует операцию активации **Rectified Linear Unit** (ReLU), которая является одной из самых популярных функций активации в нейронных сетях. Она используется для добавления нелинейности в модель и помогает бороться с проблемой исчезающих градиентов. ReLU возвращает ноль для всех входных значений, которые меньше или равны нулю, и сам вход, если он больше нуля.


  Функция активации ReLU возвращает значение входа, если оно больше нуля, и ноль в противном случае.
  
  $$ \text{output} = \max(\text{input}, 0) $$


  Градиенты сохраняются только для тех элементов входа, которые больше нуля. Если входное значение меньше или равно нулю, то соответствующий градиент становится равным нулю.

  $$ \text{gradInput} = \text{gradOutput} \cdot (input > 0) $$


In [11]:
class ReLU(Module):
    def __init__(self):
        super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        # Для ReLU активации выходные данные равны входу, если вход больше нуля, и 0 в противном случае.
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Метод обновления градиентов по входным данным
        # Градиенты сохраняются, только если соответствующий вход больше нуля (для ReLU).
        # Если вход меньше или равен нулю, градиент становится нулевым.
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        # Представление объекта класса ReLU
        return "ReLU"

## Criterion

Класс `Criterion` является базовым дляласс `Criterion` является базовым для реализации функций потерь в машинном обучении, обычно используется в контексте нейронных сетей. Он предоставляет основную структуру для вычисления прямого прохода (потерь) и обратного прохода (градиентов), что является необходимым для обучения моделей с использованием градиентного спуска.



In [12]:
class Criterion(object):
    def __init__(self):
        self.output = None  # Результат функции потерь
        self.gradInput = None  # Градиенты функции потерь

    # Метод для вычисления функции потерь (функция прямого распространения)
    def forward(self, input, target):
        """
            Принимает входные данные (input) и целевые значения (target),
            вычисляет функцию потерь, связанную с этим критерием, и возвращает результат.

            Для согласованности эта функция не должна переопределяться,
            весь код должен быть в методе `updateOutput`.
        """
        return self.updateOutput(input, target)

    # Метод для вычисления градиентов (функция обратного распространения)
    def backward(self, input, target):
        """
            Принимает входные данные (input) и целевые значения (target),
            вычисляет градиенты функции потерь, связанные с этим критерием, и возвращает результат.

            Для согласованности эта функция не должна переопределяться,
            весь код должен быть в методе `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    # Метод для вычисления потерь(input, target)
    
    # Метод для вычисления потерь (предназначен для переопределения в дочерних классах)
    def updateOutput(self, input, target):
        """
        Функция для переопределения в дочерних классах.
        """
        return self.output

    # Метод для вычисления градиентов (предназначен для переопределения в дочерних классах)
    def updateGradInput(self, input, target):
        """
        Функция для переопределения в дочерних классах.
        """
        return self.gradInput   

    # Метод для строкового представления объекта (для удобства вывода)
    def __repr__(self):
        """
        Преобразование объекта в строку. Должен быть переопределен в каждом модуле,
        если необходимо получить читаемое описание.
        """
        return "Criterion"

**`ClassNLLCriterion`**

Класс `ClassNLLCriterion` реализует **функцию потерь на основе негативного логарифма правдоподобия (NLL)** для многоклассовой классификации. Эта функция потерь часто используется в задачах классификации, когда модель должна предсказать вероятность для каждого класса. В этом классе функция потерь минимизируется с использованием кросс-энтропии, которая позволяет модели лучше предсказывать правильный класс, минимизируя ошибку.

**Зачем это нужно?**
`ClassNLLCriterion` используется для вычисления потерь и градиентов в многоклассовых задачах классификации. Его основная цель — минимизировать разницу между предсказанными вероятностями для правильного класса и истинными метками классов. Этот класс необходим для обучения нейронных сетей с использованием алгоритма обратного распространения ошибки (backpropagation).


In [13]:
class ClassNLLCriterion(Criterion):
    """
    Реализация критерия Class Negative Log-Likelihood (NLL).
    Этот критерий используется для задач классификации, где входные данные (input)
    представляют собой логарифмы вероятностей классов. Он вычисляет
    отрицательный логарифм вероятности правильного класса и возвращает
    среднее значение по всем примерам в мини-партии.
    """
    def __init__(self):
        """
        Инициализирует критерий ClassNLLCriterion.
        """
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):

        batch_size, n_classes = input.shape

        # Находим индексы правильных классов из one-hot encoding
        target_indices = np.argmax(np.asarray(target), axis=1).astype(int)

        # Извлекаем логарифмы вероятностей правильных классов
        correct_class_log_probs = input[np.arange(batch_size), target_indices]

        # Вычисляем средний отрицательный логарифм правдоподобия
        self.output = -np.mean(correct_class_log_probs)
        return self.output

    def updateGradInput(self, input, target):
        batch_size, n_classes = input.shape

        # Находим индексы правильных классов из one-hot encoding
        target_indices = np.argmax(np.asarray(target), axis=1).astype(int)

        # Инициализируем градиент нулями
        self.gradInput = np.zeros_like(input)

        # Устанавливаем градиент равным -1.0 для правильных классов
        self.gradInput[np.arange(batch_size), target_indices] = -1.0

        # Нормализуем градиент, разделив на размер мини-партии
        self.gradInput /= batch_size
        return self.gradInput

    def __repr__(self):
        """
        Возвращает строковое представление критерия.
        """
        return "ClassNLLCriterion"

Класс `MSECriterion` реализует функцию потерь для задачи регрессии — **среднеквадратичную ошибку (MSE)**. Эта функция используется для измерения отклонений между предсказанными значениями и реальными целевыми значениями. MSE вычисляется как среднее квадратичное отклонение между входными и целевыми значениями.

  - Этот метод вычисляет значение функции потерь (среднеквадратичной ошибки) для заданных входных данных (`input`) и целевых значений (`target`).
  - Математически MSE определяется как:
  $$
  MSE = \frac{1}{N} \sum_{i=1}^{N} (input_i - target_i)^2
  $$
  где $N$ — количество примеров в батче.

  - Метод для вычисления градиентов функции потерь по входным данным, который используется при обратном распространении ошибки.
  - Градиенты вычисляются как:
  $$
  \frac{\partial MSE}{\partial input_i} = \frac{2}{N} (input_i - target_i)
  $$



In [14]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    # Метод для вычисления функции потерь (в данном случае, среднеквадратичной ошибки)
    def updateOutput(self, input, target):   
        # Вычисляем среднеквадратичную ошибку (MSE)
        self.output = np.sum(np.power(input - target, 2)) / input.shape[0]
        return self.output 

    # Метод для вычисления градиентов функции потерь (для обратного распространения ошибки)
    def updateGradInput(self, input, target):
        # Вычисляем градиенты функции потерь по входным данным
        # Нормализуем по размеру батча
        self.gradInput = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"


## Optimizers

### Оптимизатор SGD с моментумом

Оптимизатор **SGD с моментумом** (Stochastic Gradient Descent with Momentum) используется для улучшения процесса оптимизации, добавляя моментум для ускорения движения в направлении минимизации функции потерь.


Для каждого параметра (веса или смещения) оптимизатор обновляет значение по следующей формуле:

$$
v_t = \beta v_{t-1} + (1 - \beta) \nabla \theta_t
$$

где:
- $v_t$ — это накопленный моментум в момент времени $t$.
- $\beta$ — коэффициент моментума (например, 0.9).
- $\nabla \theta_t$ — это градиент функции потерь по параметру в момент времени $t$.

Затем параметры обновляются по формуле:

$$
\theta_t = \theta_{t-1} - \eta v_t
$$

где:
- $\eta$ — это скорость обучения (learning rate).
- $\theta_t$ — это значение параметра после обновления.

1. Для каждого слоя:
   1. Обновить накопленные градиенты с помощью формулы для $v_t$.
   2. Обновить параметры слоя с использованием обновленных накопленных градиентов $v_t$.
   
2. Повторять эти шаги для каждого мини-батча в процессе обучения.
.


In [15]:
def sgd_momentum(variables, gradients, config, state):  

    state.setdefault('accumulated_grads', {})

    var_index = 0
    
    # Проходим по каждому слою сети (или другой структуре данных) и его градиентам.
    for current_layer_vars, current_layer_grads in zip(variables, gradients): 
        # Для каждой переменной в слое и соответствующего ей градиента:
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            
            # Получаем старый градиент, если он уже накоплен, или создаем новый (независимо от старого значения).
            old_grad = state['accumulated_grads'].setdefault(var_index, np.zeros_like(current_grad))
            
            # Обновляем накопленный градиент с учетом инерции (моментума) и текущего градиента.
            np.add(config['momentum'] * old_grad, config['learning_rate'] * current_grad, out=old_grad)
            
            # Обновляем текущую переменную (параметр) с учетом инметр) с учетом инкремента, определяемого моментумом и градиентом.
            current_var -= old_grad
            var_index += 1  # Увеличиваем индекс переменной для отслеживания следующей переменной.



### Оптимизатор Adam

Оптимизатор Adam (Adaptive Moment Estimation) является популярным методом оптимизации, который комбинирует преимущества двух других методов: адаптивного градиентного спуска (AdaGrad) и метода моментума. Он эффективно использует информацию о первом и втором моменте градиентов для адаптации шагов обучения.

**Формулы для работы оптимизатора:**

1. **Текущий шаг обучения**:
   $$ lr_t = \text{learning\_rate} * \frac{\sqrt{1-\beta_2^t}}{1-\beta_1^t} $$

2. **Первый момент (экспоненциально сглаженный градиент)**:
   $$ \mu_t = \beta_1 * \mu_{t-1} + (1 - \beta_1)*g $$

3. **Второй момент (экспоненциально сглаженная квадратичная ошибка градиента)**:
   $$ v_t = \beta_2 * v_{t-1} + (1 - \beta_2)*g^2 $$

4. **Обновление значений переменных**:
   $$ \text{variable} = \text{variable} - \text{lr}_t * \frac{m_t}{\sqrt{v_t} + \epsilon} $$

Adam адаптирует скорость обучения для каждой переменной, улучшая эффективность оптимизации, особенно в случае с шумными градиентами или на малых данных.


In [16]:
def adam(variables, gradients, config, state):  
    state.setdefault('m', {})
    state.setdefault('v', {})
    state.setdefault('t', 0)   # инициализация счетчика времени t как 0, если его нет в state
    state['t'] += 1  # увеличиваем счетчик времени (номер итерации)
    
    var_index = 0
    # Адаптированный шаг обучения с учетом временного сглаживания
    lr_t = config['learning_rate'] * np.sqrt(1 - config['beta2']**state['t']) / (1 - config['beta1']**state['t'])
    
    # Итерация по слоям переменных и градиентов
    for current_layer_vars, current_layer_grads in zip(variables, gradients): 
        # Итерация по переменным и градиентам текущего слоя
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            # Инициализация первого момента (m) для текущ # Инициализация первого момента (m) для текущей переменной, если он еще не существует
            var_first_moment = state['m'].setdefault(var_index, np.zeros_like(current_grad))
            # Инициализация второго момента (v) для текущей переменной, если он еще не существует
            var_second_moment = state['v'].setdefault(var_index, np.zeros_like(current_grad))
            
            # Обновление первого момента: экспоненциально сглаженный градиент
            np.add(config['beta1'] * var_first_moment, (1 - config['beta1']) * current_grad, out=var_first_moment)
            
            # Обновление второго момента: экспоненциально сглаженная квадратичная ошибка
            np.add(config['beta2'] * var_second_moment, (1 - config['beta2']) * np.square(current_grad), out=var_second_moment)
            
            # Обновление значения переменной с учетом адаптированного шага обучения
            current_var -= lr_t * var_first_moment / (np.sqrt(var_second_moment) + config['epsilon'])
            
            # Проверка, что состояния первого и второго моментов обновлены корректно
            assert var_first_moment is state['m'].get(var_index)
            assert var_second_moment is state['v'].get(var_index)
            
            var_index += 1  # Переход к следующей переменной в слое


## Conv2D - Двумерный сверточный слой

Реализация классического сверточного слоя для обработки изображений и других 2D данных.

`Сверточный слой (Conv2D)` - это основной строительный блок сверточных нейронных сетей (CNN), который применяет операцию свертки к входным данным, извлекая локальные признаки и сохраняя пространственные отношения.


In [17]:
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2d, self).__init__()
        assert kernel_size % 2 == 1, kernel_size  # Проверка, что размер ядра нечетный
       
        stdv = 1./np.sqrt(in_channels)  # Стандартное отклонение для инициализации весов
        # Инициализация весов случайными числами в диапазоне от -stdv до stdv
        self.W = np.random.uniform(-stdv, stdv, size=(out_channels, in_channels, kernel_size, kernel_size))
        # Инициализация смещений
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        # Инициализация градиентов
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        pad_size = self.kernel_size // 2  # Размер паддинга (для симметричного паддинга)
        
        batch_size, channels, height, width = input.shape
        # Паддинг входа (добавляем нули вокруг каждого изображения)
        padded_input = np.pad(input, ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)), mode='constant')
        
        self.output = np.zeros((batch_size, self.out_channels, height, width))  # Выходная матрица
        
        for b in range(batch_size):  # Для каждого батча
            for out_ch in range(self.out_channels):  # Для каждого выходного канала
                for in_ch in range(self.in_channels):  # Для каждого входного канала
                    # Применение свертки для соответствующих входного канала и выходного канала
                    self.output[b, out_ch] += sp.signal.correlate(
                        padded_input[b, in_ch], 
                        self.W[out_ch, in_ch], 
                        mode='valid'
                    )
                # Добавление смещения к выходу
                self.output[b, out_ch] += self.b[out_ch]
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        pad_size = self.kernel_size // 2  # Размер паддинга для градиента
        
        batch_size, in_channels, height, width = input.shape
        
        self.gradInput = np.zeros_like(input)  # Инициализация градиента входа
        
        for b in range(batch_size):  # Для каждого батча
            for in_ch in range(self.in_channels):  # Для каждого входного канала
                for out_ch in range(self.out_channels):  # Для каждого выходного канала
                    flipped_kernel = np.rot90(self.W[out_ch, in_ch], 2)  # Отражение ядра для обратной свертки
                    
                    # Паддинг градиента выходного тензора
                    padded_grad = np.pad(gradOutput[b, out_ch], 
                                         ((pad_size, pad_size), 
                                          (pad_size, pad_size)), 
                                         mode='constant')
                    
                    # Корреляция паддинга градиента с отражённым ядром
                    self.gradInput[b, in_ch] += sp.signal.correlate(
                        padded_grad,
                        flipped_kernel,
                        mode='valid'
                    )
        
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        pad_size = self.kernel_size // 2  # Размер паддинга для градиента
        
        batch_size, in_channels, height, width = input.shape
        # Паддинг входа
        padded_input = np.pad(input, ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)), mode='constant')
        
        # Вычисление градиентов для весов
        for out_ch in range(self.out_channels):  # Для каждого выходного канала
            for in_ch in range(self.in_channels):  # Для каждого входного канала
                for b in range(batch_size):  # Для каждого батча
                    self.gradW[out_ch, in_ch] += sp.signal.correlate(
                        padded_input[b, in_ch],  # Корреляция входного тензора с градиентом выхода
                        gradOutput[b, out_ch],
                        mode='valid'
                    )
        
        self.gradb = np.sum(gradOutput, axis=(0, 2, 3))  # Суммируем градиенты по всем осям

    def zeroGradParameters(self):
        # Обнуляем градиенты
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        # Получение параметров (веса и смещения)
        return [self.W, self.b]
    
    def getGradParameters(self):
        # Получение градиентов параметров
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Conv2d %d -> %d' % (s[1], s[0])
        return q

## MaxPool2d

Этот класс реализует операцию двумерного максимального пулинга (`MaxPooling2d`). Пулинг — это операция, часто используемая в сверточных нейронных сетях для уменьшения размерности данных, улучшения обобщающей способности модели и повышения устойчивости к небольшим искажениям.


In [18]:
class MaxPool2d(Module):
    def __init__(self, kernel_size):
        super(MaxPool2d, self).__init__()  # Инициализация родительского класса
        self.kernel_size = kernel_size  # Размер ядра (фильтра)
        self.gradInput = None  # Здесь будет храниться градиент для входных данных

    def updateOutput(self, input):
        # Извлекаем размеры входного тензора: батч, каналы, высота и ширина
        batch_size, n_channels, input_h, input_w = input.shape

        # Убеждаемся, что размеры входа кратны размеру ядра
        assert input_h % self.kernel_size == 0  
        assert input_w % self.kernel_size == 0
        
        # Определяем параметры для вычислений
        KH, KW = (self.kernel_size, self.kernel_size)  # Высота и ширина ядра
        S = self.kernel_size  # Шаг (stride), который равен размеру ядра

        # Вычисляем размеры выходного тензора
        H_out = input_h // self.kernel_size
        W_out = input_w // self.kernel_size

        # Инициализируем выходной тензор и массив для хранения индексов максимальных значений
        self.output = np.zeros((batch_size, n_channels, H_out, W_out))
        self.max_indices = np.zeros_like(self.output, dtype=int)

        # Проходим по всем батчам и каналам
        for n in range(batch_size):
            for c in range(n_channels):
                for h in range(H_out):
                    for w in range(W_out):
                        # Определяем область для пулинга
                        h_start, h_end = h * S, h * S + KH
                        w_start, w_end = w * S, w * S + KW
                        
                        # Извлекаем окно значений для пулинга
                        window = input[n, c, h_start:h_end, w_start:w_end]

                        # Находим максимальное значение в окне
                        self.output[n, c, h, w] = np.max(window)

                        # Находим индекс максимального значения в окне (плоский индекс)
                        max_idx_flat = np.argmax(window)
                        # Сохраняем этот индекс
                        self.max_indices[n, c, h, w] = max_idx_flat

        # Возвращаем выходной тензор
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Извлекаем размеры входных данных
        batch_size, n_channels, H_in, W_in =  input.shape
        # Размеры выходного градиента
        H_out, W_out = gradOutput.shape[-2:]

        KH, KW = (self.kernel_size, self.kernel_size)  # Размеры ядра
        S = self.kernel_size  # Шаг пулинга

        # Инициализируем градиент входных данных
        self.gradInput = np.zeros(input.shape)

        # Проходим по всем батчам и каналам
        for n in range(batch_size):
            for c in range(n_channels):
                for h in range(H_out):
                    for w in range(W_out):
                        # Градиент для текущего выходного элемента
                        grad = gradOutput[n, c, h, w]
                        # Индекс максимального элемента в окне для текущего пикселя
                        max_idx_flat = self.max_indices[n, c, h, w]

                        # Преобразуем плоский индекс в двумерные координаты (для высоты и ширины)
                        max_h_rel, max_w_rel = np.unravel_index(max_idx_flat, (KH, KW))

                        # Вычисляем абсолютные координаты максимального элемента в исходном изображении
                        h_start = h * S
                        w_start = w * S
                        max_h_abs = h_start + max_h_rel
                        max_w_abs = w_start + max_w_rel

                        # Добавляем градиент к соответствующему элементу в входных данных
                        self.gradInput[n, c, max_h_abs, max_w_abs] += grad

        # Возвращаем градиент входных данных
        return self.gradInput
    
    def __repr__(self):
        q = 'MaxPool2d, kern %d, stride %d' %(self.kernel_size, self.kernel_size)
        return q
